In [1]:
import pandas as pd
import datetime
from sklearn.mixture import BayesianGaussianMixture
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
path = 'ad_ranking_raw_data.xlsx'

ads = pd.read_excel(path, sheet_name='ads dimension (dim table)', header=1)
mods = pd.read_excel(path, sheet_name='moderator dimension (dim table)')

In [3]:
# ads
ads['avg_ad_revenue'].isna().sum()
max(ads['latest_punish_begin_date'])
ads['p_date_new'] = pd.to_datetime(ads['p_date'], format='%Y%m%d')
# ads
# ads['p_date'].nunique()

In [4]:
ads.ad_revenue = ads['ad_revenue'].fillna(0)

Fill in missing values for punish number based on inferred advertisers.

In [5]:
# ads.groupby('avg_ad_revenue')['punish_num'].fillna(ads.groupby('avg_ad_revenue')['punish_num'].agg('mode'))
# lazy grouping
groups = ads.groupby('avg_ad_revenue')

# where all the rows within a group is NaN
all_na = groups['punish_num'].transform(lambda x: x.isna().all())

# fill global mode
ads.loc[all_na, 'punish_num'] = ads['punish_num'].mode()[0]

# fill with local mode
mode_by_group = groups['punish_num'].transform(lambda x: x.mode()[0])
ads['punish_num'] = ads['punish_num'].fillna(mode_by_group)
ads['start_time'] = ads['start_time'].fillna(datetime.datetime(2000, 1, 1, 0))
# mode = ads.groupby('avg_ad_revenue')['punish_num'].agg('mode')
# mode

In [6]:
# Feature Engineering
ads['days_since_last_penalty'] = (ads['p_date_new'] - ads['latest_punish_begin_date']).dt.days
ads['days_since_start_time'] = (ads['p_date_new'] - ads['start_time']).dt.days
ads['revenue_ratio'] = (ads['ad_revenue'] / (ads['avg_ad_revenue'])).fillna(0)


In [7]:
ads

,p_date,ad_id,delivery_country,queue_market,punish_num,latest_punish_begin_date,ad_revenue,avg_ad_revenue,start_time,baseline_st,product_line,task_type_en,p_date_new,days_since_last_penalty,days_since_start_time,revenue_ratio
0,20230807,1747578422390810,US,USCA,1.0,2023-07-24,5132.1700,4795.250,2022-10-24 14:36:00,1.78,Auction Ads,auction_relation,2023-08-07,14,286,1.070261
1,20230807,1758543228094480,SA,MENA,1.0,2023-06-14,1976.2500,1063.767,2023-02-22 15:50:00,1.67,RIE,Ad Group Recall (Merged),2023-08-07,54,165,1.857785
2,20230807,1738303151826990,AU,AU,2.0,2023-07-24,1.7355,5.365,2022-07-14 05:00:00,1.84,Auction Ads,auction_relation,2023-08-07,14,388,0.323486
3,20230807,1738303920938010,NZ,NZ,2.0,2023-07-24,0.0000,5.365,2022-07-14 05:16:00,1.58,Auction Ads,auction_relation,2023-08-07,14,388,0.000000
4,20230807,1738304016410620,NZ,NZ,2.0,2023-07-24,1.5250,5.365,2022-07-14 05:17:00,1.58,Auction Ads,auction_relation,2023-08-07,14,388,0.284250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40674,20230807,1773472206047230,VN,NaN,1.0,2000-01-01,0.0000,0.000,2000-01-01 00:00:00,0.54,Non-Auction Ads,Promote,2023-08-07,8619,8619,0.000000
40675,20230807,1773607712030720,BR,NaN,1.0,2000-01-01,0.0000,0.000,2000-01-01 00:00:00,0.54,Non-Auction Ads,Promote,2023-08-07,8619,8619,0.000000
40676,20230807,1773589699308540,BE,NaN,1.0,2000-01-01,0.0000,0.000,2000-01-01 00:00:00,0.54,Non-Auction Ads,Promote,2023-08-07,8619,8619,0.000000
40677,20230807,1773530511220730,TR,NaN,1.0,2000-01-01,0.0000,0.000,2000-01-01 00:00:00,0.54,Non-Auction Ads,Promote,2023-08-07,8619,8619,0.000000


In [8]:
data = ads[['ad_id', 'delivery_country', 'queue_market', 'baseline_st', 'punish_num', 'ad_revenue', 'avg_ad_revenue', 'days_since_last_penalty', 'days_since_start_time']]

In [9]:
data.values

array([[1747578422390810, 'US', 'USCA', ..., 4795.25, 14, 286],
       [1758543228094480, 'SA', 'MENA', ..., 1063.767, 54, 165],
       [1738303151826990, 'AU', 'AU', ..., 5.365, 14, 388],
       ...,
       [1773589699308540, 'BE', nan, ..., 0.0, 8619, 8619],
       [1773530511220730, 'TR', nan, ..., 0.0, 8619, 8619],
       [1773608138524670, 'BR', nan, ..., 0.0, 8619, 8619]], dtype=object)

In [10]:
data.drop(columns=['delivery_country', 'queue_market']).describe()

,ad_id,baseline_st,punish_num,ad_revenue,avg_ad_revenue,days_since_last_penalty,days_since_start_time
count,4.067900e+04,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000,40679.000000
mean,1.772982e+15,1.206670,1.008653,54.780009,23.887703,8113.368200,1419.723395
std,2.562183e+12,0.659845,0.207590,421.777514,187.580436,2022.510702,3189.860466
min,1.697828e+15,0.540000,1.000000,0.000000,0.000000,0.000000,-38.000000
25%,1.773396e+15,0.540000,1.000000,0.000000,0.835250,8619.000000,-1.000000
50%,1.773532e+15,1.400000,1.000000,2.233600,2.597900,8619.000000,0.000000
75%,1.773564e+15,1.780000,1.000000,12.820750,8.269100,8619.000000,10.000000
max,1.773617e+15,7.590000,16.000000,27100.620000,17144.498600,8619.000000,8619.000000


In [11]:
def compute_risk_scores(X, gmm):
    responsibilities = gmm.predict_proba(X)
    risk_scores = np.max(responsibilities, axis=1)
    return risk_scores

In [12]:

# Your custom loss function
def loss(risk_scores):
    # Implement your loss function here based on the given risk scores
    return sum([risk_scores[i] for i in range(1000)])


In [13]:
scaler = StandardScaler()
X = scaler.fit_transform(data.drop(columns=['delivery_country', 'queue_market', 'ad_id']).values)

best_loss = -float('inf')  # or -float('inf') if you're maximizing
best_gmm = None
tolerance = 1e-5
max_iterations = 10000
previous_loss = best_loss
# n_components = 3

for iteration in range(max_iterations):
    gmm = BayesianGaussianMixture()
    gmm.fit(X)
    
    current_risk_scores = compute_risk_scores(X, gmm)
    current_loss = loss(current_risk_scores)

    if abs(current_loss - previous_loss) < tolerance:
        break
    
    if current_loss > best_loss:  # Use > if you're maximizing the loss
        best_loss = current_loss
        best_gmm = gmm

    previous_loss = current_loss

# best_gmm now contains the model with the lowest observed loss
final_risk_scores = compute_risk_scores(X, best_gmm)

In [14]:
pd.Series(final_risk_scores).describe()

count    40679.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
dtype: float64

In [15]:
best_gmm.aic(X)

AttributeError: 'BayesianGaussianMixture' object has no attribute 'aic'

In [ ]:
best_gmm.bic(X)

-78518.12323828928